# Preparing the dataset
Objective: Find the affected area, crop the image or fill it to make all of them have the same size. 

In [121]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import os 

# essential functions

def save_image(name, img):
    cv.imwrite('..\datasets\ISIC2017\\thresholding_experiments\\{}.jpg'.format(name), img)

def read_image(name):
    img = cv.imread('..\datasets\ISIC2017\ISIC-2017_Training_Data\{}.jpg'.format(name))
    return img

all_images = []

Obtaining all images name

In [122]:
TOTAL_IMAGES_TRAINING = 15295

all_files = os.listdir('..\datasets\ISIC2017\ISIC-2017_Training_Data\\')

for file in all_files:
        if file.find('superpixels') == -1 and file.find('Training_Data_metadata') == -1:
                new_img = file.split('.')[0]
                all_images.append(new_img)


Obtain the threshold image for gray scale image and blue channel

In [123]:
def threshold_image (img):
    img = cv.GaussianBlur(img,(5,5),0)
    _,img = cv.threshold(img,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)
    kernel = np.ones((10,10),np.uint8)
    img = cv.erode(img,kernel,iterations = 1) 
    return img

Remove black corners remaining from the lens of the camera used to take the lesion image 

In [124]:
def remove_corners(img):
    y, x = img.shape

    corners = {
        'top_left': {
            'x': [0,10],
            'y': [0,10]
        },
        'top_right': {
            'x': [0,10],
            'y': [y - 11, y-1]
        },
        'bottom_left': {
            'x': [x - 11, x-1],
            'y': [0,10]
        },
        'bottom_right': {
            'x': [x - 11, x-1],
            'y': [y - 11, y-1]
        },
    }

    for position in corners:
        for x in range(corners[position]['x'][0], corners[position]['x'][1]):
            for y in range(corners[position]['y'][0], corners[position]['y'][1]):
                if img[y,x] >= 255:
                    cv.floodFill(img, None, (x,y), 0)
    return img

Add gray and blue images

In [125]:
def add_gray_and_blue(img_1, img_2):
    img_sum = img_1 + img_2
    img_sum[img_sum > 125] = 255
    img_sum[img_sum < 125] = 0
    return img_sum
    

Remove minimal blobs from the image

In [126]:
def remove_minimal_blobs(img):
    contours, _ = cv.findContours(img, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    img_clean = np.zeros_like(img)
    for contour in contours:
        area = cv.contourArea(contour)
        if area > 5000:
            cv.fillPoly(img_clean, [contour], 255)

    kernel = np.ones((10,10),np.uint8)
    img_clean = cv.dilate(img_clean,kernel,iterations = 1)
    return img_clean

Draw rectangle arround lesion

In [127]:
def draw_rectangle(img):
    contours, _ = cv.findContours(img,cv.RETR_TREE,cv.CHAIN_APPROX_NONE)

    y, x = img.shape
    
    t, b, r, l = [-1, 10000, -1, 10000]
    
    for points in contours:
        for p in points:
            if t < p[0][1]:
                t = p[0][1]
            if b > p[0][1]:
                b = p[0][1]
            if l > p[0][0]:
                l = p[0][0]
            if r < p[0][0]:
                r = p[0][0]

    b = b - 100 if b - 100 > -1 else 0
    t = t + 100 if t + 100 < y else y
    l = l - 100 if l - 100 > -1 else 0
    r = r + 100 if r + 100 < x else x
                
    img_final = cv.rectangle(img_rgb, (l,b), (r,t), (255, 0, 0), 1)

    return img_final

In [128]:
for img_name in all_images:
    img_rgb = read_image(img_name)

    img_gray = cv.cvtColor(img_rgb, cv.COLOR_BGR2GRAY)
    img_blue = img_rgb[:,:,0]

    img_thresh_gray = threshold_image(img_gray)
    img_thresh_blue = threshold_image(img_blue)
    
    # save_image(img_name+'.jpg', img_rgb)
    # save_image(img_name+'_thresh_gray.jpg', img_thresh_gray)
    # save_image(img_name+'_thresh_blue.jpg', img_thresh_blue)

    img_gray_no_corners = remove_corners(img_thresh_gray)
    img_blue_no_corners = remove_corners(img_thresh_blue)

    # save_image(img_name+'_gray_no_corners.jpg', img_gray_no_corners)
    # save_image(img_name+'_blue_no_corners.jpg', img_blue_no_corners)

    img_sum = add_gray_and_blue(img_gray_no_corners, img_blue_no_corners)

    img_clean = remove_minimal_blobs(img_sum)
    
    img_final = draw_rectangle(img_clean)

    save_image(img_name+'final.jpg', img_final)

KeyboardInterrupt: 

In [ ]:
# for img_name in images_name:
#     img_rgb = cv.imread('..\datasets\ISIC2017\ISIC-2017_Training_Data\{}.jpg'.format(img_name))
#     img = cv.cvtColor(img_rgb, cv.COLOR_BGR2GRAY)
#     img = cv.GaussianBlur(img,(5,5),0)
#     _,img = cv.threshold(img,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)
#     kernel = np.ones((10,10),np.uint8)
#     img = cv.erode(img,kernel,iterations = 1) 

#     img_blue = img_rgb[:,:,0]
#     img_blue = cv.GaussianBlur(img_blue,(11,11),0)
#     _,img_blue = cv.threshold(img_blue,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)
#     kernel = np.ones((10,10),np.uint8)
#     img_blue = cv.erode(img_blue,kernel,iterations = 1)
    
#     y, x = img.shape

#     corners = {
#         'top_left': {
#             'x': [0,10],
#             'y': [0,10]
#         },
#         'top_right': {
#             'x': [0,10],
#             'y': [y - 11, y-1]
#         },
#         'bottom_left': {
#             'x': [x - 11, x-1],
#             'y': [0,10]
#         },
#         'bottom_right': {
#             'x': [x - 11, x-1],
#             'y': [y - 11, y-1]
#         },
#     }

#     count = 0
    
#     # remove white corners from image
#     for position in corners:
#         for x in range(corners[position]['x'][0], corners[position]['x'][1]):
#             for y in range(corners[position]['y'][0], corners[position]['y'][1]):
#                 if img[y,x] >= 255:
#                     cv.floodFill(img, None, (x,y), 0)
#                 if img_blue[y,x] >= 255:
#                     cv.floodFill(img_blue, None, (x,y), 0)
    
#     img_sum = img + img_blue
#     img_sum[img_sum > 125] = 255
#     img_sum[img_sum < 125] = 0

#     #remove minimal white blobs from images
#     contours, _ = cv.findContours(img_sum, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
#     img_clean = np.zeros_like(img_sum)
#     for contour in contours:
#         area = cv.contourArea(contour)
#         if area > 5000:
#             cv.fillPoly(img_clean, [contour], 255)

    
#     img_clean = cv.dilate(img_clean,kernel,iterations = 1)

#     contours, _ = cv.findContours(img_clean,cv.RETR_TREE,cv.CHAIN_APPROX_NONE)
    
#     t, b, r, l = [-1, 10000, -1, 10000]
    
#     for points in contours:
#         for p in points:
#             if t < p[0][1]:
#                 t = p[0][1]
#             if b > p[0][1]:
#                 b = p[0][1]
#             if l > p[0][0]:
#                 l = p[0][0]
#             if r < p[0][0]:
#                 r = p[0][0]

#     b = b - 100 if b - 100 > -1 else 0
#     t = t + 100 if t + 100 < y else y
#     l = l - 100 if l - 100 > -1 else 0
#     r = r + 100 if r + 100 < x else x
    
#     print(str(t), str(b), str(r), str(l))
                
#     img_final = cv.rectangle(img_rgb, (l,b), (r,t), (255, 0, 0), 1)

#     cv.imwrite('..\datasets\ISIC2017\\thresholding_experiments\methods_comparison\\'+img_name+'_method1.5.jpg', img_final)